# Introduction to Blocks

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
# Install required libraries for Transformers, datasets, evaluation, and Gradio interface
!uv pip install datasets evaluate transformers[sentencepiece]
!uv pip install gradio

In [ ]:
# Introduction to Blocks - more flexible alternative to Interface
# Blocks allows for custom layouts, multiple functions, and complex interactions
import gradio as gr


def flip_text(x):
    # Simple function to reverse text strings
    return x[::-1]


# Create a Blocks app with custom layout and styling
demo = gr.Blocks()

with demo:
    # Add markdown content for instructions and formatting
    gr.Markdown(
        """
    # Flip Text!
    Start typing below to see the output.
    """
    )
    # Create input and output textboxes
    input = gr.Textbox(placeholder="Flip this text")
    output = gr.Textbox()

    # Set up event handling: when input changes, trigger flip_text function
    # This enables real-time text flipping as user types
    input.change(fn=flip_text, inputs=input, outputs=output)

demo.launch()

In [ ]:
# Advanced Blocks layout with tabs, rows, and multiple functions
# This demonstrates complex UI organization and multiple processing functions
import numpy as np
import gradio as gr

demo = gr.Blocks()


def flip_text(x):
    # Reverse text character by character
    return x[::-1]


def flip_image(x):
    # Flip image horizontally using numpy
    # np.fliplr flips array left-right (horizontal flip)
    return np.fliplr(x)


with demo:
    gr.Markdown("Flip text or image files using this demo.")
    
    # Create tabbed interface for different functionalities
    with gr.Tabs():
        # First tab: Text flipping functionality
        with gr.TabItem("Flip Text"):
            with gr.Row():  # Arrange components horizontally
                text_input = gr.Textbox()
                text_output = gr.Textbox()
            text_button = gr.Button("Flip")  # Manual trigger button
            
        # Second tab: Image flipping functionality  
        with gr.TabItem("Flip Image"):
            with gr.Row():  # Arrange components horizontally
                image_input = gr.Image()
                image_output = gr.Image()
            image_button = gr.Button("Flip")  # Manual trigger button

    # Set up event handlers for button clicks
    # Unlike the previous example, these require manual button clicks
    text_button.click(flip_text, inputs=text_input, outputs=text_output)
    image_button.click(flip_image, inputs=image_input, outputs=image_output)

demo.launch()

In [ ]:
# Text completion interface using external API integration
# This demonstrates how to integrate external models/APIs into Gradio Blocks
import gradio as gr

# Load external GPT-J model API through Gradio's Interface.load()
# This provides access to large language models without local hosting
api = gr.Interface.load("huggingface/EleutherAI/gpt-j-6B")


def complete_with_gpt(text):
    # Use only the last 50 characters as context for efficiency
    # This prevents overwhelming the API with very long texts
    # Concatenate original text (minus last 50 chars) with API completion
    return text[:-50] + api(text[-50:])


# Create Blocks interface for text completion
with gr.Blocks() as demo:
    textbox = gr.Textbox(placeholder="Type here and press enter...", lines=4)
    btn = gr.Button("Generate")

    # Connect button click to completion function
    # When clicked, processes textbox content and updates same textbox with result
    btn.click(complete_with_gpt, textbox, textbox)

demo.launch()

In [ ]:
# Multi-step pipeline: Speech recognition → Sentiment analysis
# This demonstrates chaining multiple AI models in a sequential workflow
from transformers import pipeline
import gradio as gr

# Load two different AI pipelines for a multi-step process
asr = pipeline("automatic-speech-recognition", "facebook/wav2vec2-base-960h")  # Speech-to-text
classifier = pipeline("text-classification")  # Sentiment analysis (default model)


def speech_to_text(speech):
    # Convert audio file to text using automatic speech recognition
    # Returns the transcribed text from the audio input
    text = asr(speech)["text"]
    return text


def text_to_sentiment(text):
    # Analyze sentiment of text input
    # Returns the predicted sentiment label (POSITIVE/NEGATIVE)
    return classifier(text)[0]["label"]


# Create multi-step interface using Blocks
demo = gr.Blocks()

with demo:
    # Create components for the pipeline
    audio_file = gr.Audio(type="filepath")  # Audio input (file path for model)
    text = gr.Textbox()                     # Intermediate text output
    label = gr.Label()                      # Final sentiment output

    # Create buttons for each step of the pipeline
    b1 = gr.Button("Recognize Speech")      # Step 1: Audio → Text
    b2 = gr.Button("Classify Sentiment")   # Step 2: Text → Sentiment

    # Connect each button to its respective function
    # This creates a manual two-step process that users control
    b1.click(speech_to_text, inputs=audio_file, outputs=text)
    b2.click(text_to_sentiment, inputs=text, outputs=label)

demo.launch()

In [ ]:
# Dynamic interface updates - demonstrates reactive UI components
# This shows how interface elements can change based on user selections
import gradio as gr


def change_textbox(choice):
    # Function to modify textbox properties based on radio button selection
    # gr.Textbox.update() allows dynamic modification of component properties
    if choice == "short":
        return gr.Textbox.update(lines=2, visible=True)      # Small textbox
    elif choice == "long":
        return gr.Textbox.update(lines=8, visible=True)      # Large textbox  
    else:
        return gr.Textbox.update(visible=False)              # Hidden textbox


# Create dynamic interface with reactive components
with gr.Blocks() as block:
    # Radio button for selecting essay type
    radio = gr.Radio(
        ["short", "long", "none"], 
        label="What kind of essay would you like to write?"
    )
    # Textbox that will change based on radio selection
    text = gr.Textbox(lines=2, interactive=True)

    # Set up reactive behavior: when radio changes, update textbox
    # This creates a dynamic user experience where UI adapts to selections
    radio.change(fn=change_textbox, inputs=radio, outputs=text)
    block.launch()